In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/Shareddrives/ECE 4554 Computer Vision: Celestial Map Generation/feature point detection')

In [ ]:
import cv2 # OpenCV library
from PIL import Image # Python Imaging Library
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

from google.colab.patches import cv2_imshow

In [ ]:
def linear_filter(img_in, kernel):
  '''Filter an input image by applying cross-correlation with a kernel.

  Input: 
    img_in: a grayscale image of any size larger than the kernel, 
     in both row and column directions.
    kernel: a 2D array of floating-point values;
     you may assume that this array is square, 
     with an odd number of rows and an odd number of columns;
     use the *center* of this kernel as its point of reference for filtering.
     Perform cross-correlation (do not reverse the kernel).

  Return value:
     an image with the same row/column size as img_in, 
     but each pixel is a floating-point value;
     apply the kernel only at locations where it fits entirely within the 
     input image; 
     the remaining pixels (near the outside border of the output image)
     must be set to zero;
     for any negative values, take the absolute value;
     clip the final output so that every pixel value lies in the range 0 to 255.

  '''

  # The following line is simply a placeholder; replace it with your code
  
  k_h,k_w = np.shape(kernel)
  m = k_h//2
  n = k_w//2
  h,w = np.shape(img_in)
  img_out = np.zeros((h,w), dtype=np.float32)
  for i in range(h-m):
    for j in range(w-n):
      current_out = 0
      for k in range(-m, m+1):
        for l in range(-n, n+1):
          current_out += img_in[i+k,j+l]*kernel[k+m,l+n]
          if current_out > 255:
            current_out = 255
      img_out[i,j] = (abs(np.float32(current_out)))
      
  return img_out # Each pixel must be of type np.float32
  

In [ ]:
def gaussian_kernel():
    """
    Return a Gaussian kernel, to be used for image filtering
        
    Input parameters: 
     none

    Return value:
     a kernel of size 7x7, 
     approximating a 2D Gaussian function with sigma = 0.63

    """

    # The following line is simply a placeholder; replace it with your code
    newkernel = np.zeros((7, 7), dtype = float)

    size = 7
    m = size//2
    n = m

    sigma = 0.63
    const = 1/(2*np.pi*(sigma**2))

    for x in range(-m, m+1):
      for y in range(-n, n+1):
        exp = -(x**2 + y**2)/(2*(sigma**2))
        newkernel[x+m,y+n] = const*pow(np.e,exp)
  
    return newkernel

In [ ]:
def subsample(img_in):
  '''Create a new image that has half the resolution as the input image
     by discarding pixels

  Input: 
    img_in: a grayscale image 

  Return value:
    img_out: a new image of half size 
  '''
  h,w = np.shape(img_in)
  h = h//2
  w = w//2
  img_out = np.zeros((h, w))

  for i in range(h):
    for j in range(w):
      img_out[i][j] = img_in[2*i][2*j]

  return img_out

In [ ]:
def dog(img):
  """
  Return the difference of the input image and the image filtered by the gaussian kernel

  Input:
  img_in: The input image

  Returns:
  dog: The differece of the input image and the image filtered by the gaussian kernel

  """
  img_in = cv2.bitwise_not(img)
  kernel = gaussian_kernel()
  blurred = linear_filter(img_in, kernel)
  diff = img_in - blurred 
  return diff

In [ ]:
def generate_dog(img_in, levels):
  current_image = subsample(img_in)
  current_level = 1
  for count in range(1, (levels + 1)):
    current_image = dog(img_in)
    print('Current Level: ', current_level )
    cv2_imshow(current_image)



In [ ]:
def cv2_dog(img_in):
  high = cv2.GaussianBlur(img_in, (9,9), cv2.BORDER_DEFAULT)
  dog = img_in - high
  return(dog)
  

In [ ]:
def cv2_generate_dog(img_in, levels):
  dogs = []
  current_image = cv2.bitwise_not(img_in)
  current_level = 0
  for count in range(1, (levels + 1)):
    print('Current Blur Level: ', current_level)
    cv2_imshow(current_image)
    difference = cv2_dog(current_image)
    current_image = cv2.GaussianBlur(current_image,  (9,9), cv2.BORDER_DEFAULT)
    current_level += 1
    print('Difference: ')
    cv2_imshow(difference)
    dogs.append(difference)

  return dogs


In [ ]:
def extract_feature_points(dog_stack, filter_level, stack_index, max_rad, min_rad):
  """
  Input:

  dog_stack: Stack of Difference of Gaussians 
  filter_level: Kernel Size for median blurring 
  stack_index: Index of Difference of Gaussian in Stack to be used for feature extraction
  max_rad: radius in pixels of largest star
  min_rad: radius in pixels of smallest star

  Output: 

  feature_points: n x 3 array of n feature points. (x, y, rad)

  """
  noise_removed = cv2.medianBlur(dog_stack[stack_index], filter_level)
  
  print('Image After', stack_index, 'Differences of Gaussians')
  cv2_imshow(dog_stack[stack_index])
  print('Salt and pepper noise removal: ')
  cv2_imshow(noise_removed)

  m, n = np.shape(noise_removed)
  plot = np.zeros((m, n, 3))

  feature_points = cv2.HoughCircles(noise_removed, cv2.HOUGH_GRADIENT, 1, 20,param1=15,param2=5,minRadius= min_rad,maxRadius=max_rad)

  circles = np.uint16(np.around(feature_points))
  for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(plot,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(plot,(i[0],i[1]),2,(0,0,255),3)
  print(np.shape(feature_points)[1], 'Feature Points Detected')
  cv2_imshow(plot)
  fp = np.squeeze(feature_points)
  
  radii = []
  points = []
  
  for f in fp:
    radii.append(f[2])
    points.append((f[0], f[1]))

  return points, radii

In [ ]:

filename_left = "IMG_3434.tif"
filename_right = "IMG_3433.tif"

img_left = cv2.imread(filename_left, cv2.IMREAD_GRAYSCALE)
img_right = cv2.imread(filename_right, cv2.IMREAD_GRAYSCALE)





In [ ]:
dog_stack_left = cv2_generate_dog(img_left, 10)
dog_stack_right = cv2_generate_dog(img_right, 10)

In [ ]:
feature_points_left, labels_left = extract_feature_points(dog_stack_left, 9, 9, 20, 4)
feature_points_right, labels_right = extract_feature_points(dog_stack_right, 9, 9, 20, 4)

In [ ]:
print(feature_points_left)

[(3608.5, 863.5), (5463.5, 2270.5), (1207.5, 89.5), (1469.5, 4005.5), (3051.5, 2576.5), (4991.5, 3477.5), (1203.5, 1750.5), (3654.5, 1061.5), (4568.5, 510.5), (2580.5, 589.5), (1423.5, 1177.5), (1299.5, 2524.5), (3013.5, 176.5), (2952.5, 2135.5), (5390.5, 1542.5), (4251.5, 2493.5), (3419.5, 2475.5), (1950.5, 2979.5), (1036.5, 4126.5), (2393.5, 3926.5), (2401.5, 2007.5), (272.5, 3287.5), (4369.5, 1207.5), (2256.5, 3375.5), (774.5, 613.5), (2526.5, 359.5), (6023.5, 3551.5), (1454.5, 1443.5), (4372.5, 3214.5), (1104.5, 2481.5), (6218.5, 1588.5), (1458.5, 69.5), (4864.5, 2704.5), (1970.5, 4054.5), (2284.5, 768.5), (2113.5, 1511.5), (3612.5, 3670.5), (802.5, 3941.5), (2596.5, 4108.5), (1516.5, 378.5), (1905.5, 3664.5), (5399.5, 1372.5), (2335.5, 3416.5), (3331.5, 676.5), (789.5, 1760.5), (4217.5, 3663.5), (3179.5, 644.5), (6204.5, 1682.5), (1609.5, 1188.5), (1810.5, 4130.5), (5419.5, 3636.5), (5077.5, 2528.5), (56.5, 2582.5), (237.5, 1441.5), (2936.5, 1451.5), (6087.5, 2167.5), (3361.5, 388

In [ ]:
def get_NearestNeighbors(index, features, n):
  current_star = features[index]
  nearest_neighbors = []
  i = 0
  for feature in features:
    a = np.array([current_star[0], current_star[1], 0])
    b = np.array([feature[0], feature[1], 0])
    distance = np.linalg.norm(a - b)
    nearest_neighbors.append((int(distance), i))
    i += 1
  nearest_neighbors = sorted(nearest_neighbors)
  return((nearest_neighbors[: 25]))
  
get_NearestNeighbors(0, feature_points_left, 25)

In [ ]:

def match_features(features_src, features_dst):
 
  n_neighbors = 25

  #left neighbors
  n_features_left = len(features_src[:])
 
  left_neighbors = np.zeros(( n_features_left,n_neighbors))
  left_distances = np.zeros(( n_features_left,n_neighbors))
  #print(left_neighbors)
  for i in range(n_features_left):
    ln = np.array(get_NearestNeighbors(i, feature_points_left, 25))
    #print("ln",ln,'/n')
    #ln outputs the array of 25 2D-Tuples per Star that shows the nearest neighbor and its distance
  
    for j in range(n_neighbors):

      left_neighbors[i][j]= ln[j][1]
      left_distances[i][j]= ln[j][0]
   
  #right neighbors
  n_features_right = len(features_dst[:])
 
  right_neighbors = np.zeros(( n_features_right,n_neighbors))
  right_distances = np.zeros(( n_features_right,n_neighbors))
  #print(left_neighbors)
  for i in range(n_features_right):
    ln = np.array(get_NearestNeighbors(i, feature_points_right, 25))
    #print("ln",ln,'/n')
    #ln outputs the array of 25 2D-Tuples per Star that shows the nearest neighbor and its distance
  
    for j in range(n_neighbors):

      right_neighbors[i][j]= ln[j][1]
      right_distances[i][j]= ln[j][0]
  #do a repeat of the same thing for right
  print(np.shape(left_neighbors))
  print("neighbors",'/n',left_neighbors,'/n')
  print("distances",'/n',left_distances,'/n')

  print(np.shape(right_neighbors))
  print("right neighbors",'/n',right_neighbors,'/n')
  print("right distances",'/n',right_distances,'/n')
  ######################################################
  #Now its time to find the matches for the  neighbors
  # 
  rand_left= np.rand
match_features(feature_points_left,feature_points_right)

(495, 25)
neighbors /n [[  0.  87. 295. ... 208. 459. 165.]
 [  1. 275. 443. ...  58. 287. 212.]
 [  2. 199.  31. ... 169. 355. 354.]
 ...
 [492. 365. 342. ... 445. 477. 407.]
 [493. 305. 450. ... 143. 371. 209.]
 [494. 327.  62. ... 195.  65. 171.]] /n
distances /n [[   0.   72.   84. ...  629.  635.  636.]
 [   0.   73.  139. ...  599.  616.  622.]
 [   0.  228.  251. ... 1007. 1012. 1025.]
 ...
 [   0.   67.  122. ...  502.  506.  513.]
 [   0.  327.  349. ...  748.  779.  781.]
 [   0.  164.  229. ...  626.  627.  631.]] /n
(547, 25)
right neighbors /n [[  0. 287. 230. ... 298. 229. 286.]
 [  1. 386. 279. ... 151. 463. 502.]
 [  2. 205. 397. ... 316. 445. 512.]
 ...
 [544. 142. 345. ... 344. 298. 113.]
 [545. 524.  84. ... 507. 388. 444.]
 [546. 436. 439. ... 258. 197.  88.]] /n
right distances /n [[  0. 164. 169. ... 476. 492. 514.]
 [  0. 122. 151. ... 638. 646. 687.]
 [  0.  70. 145. ... 539. 544. 547.]
 ...
 [  0. 170. 229. ... 476. 483. 509.]
 [  0. 130. 158. ... 808. 824. 840

In [ ]:
from sklearn.neighbors import NearestNeighbors
#def feature_matching(features_src, features_dst,threshold=0.7):
#for p in range(len(300)):
# X= features_src
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y= np.array([[-1, -1], [-3, -1], [-3, -4], [1, 1], [2, 1], [2, 2]])
K = 2
nbrs = NearestNeighbors(n_neighbors=K, algorithm='ball_tree').fit(X)
Xdistances, Xindices = nbrs.kneighbors(X)
distances, indices = nbrs.kneighbors(Y)
print(distances,'/n''/n')
print(" ")
print(Xdistances,'/n''/n')
#print(indices)

[[0.         1.        ]
 [1.         1.        ]
 [2.         3.16227766]
 [0.         1.        ]
 [0.         1.        ]
 [1.         1.        ]] /n/n
 
[[0.         1.        ]
 [0.         1.        ]
 [0.         1.41421356]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.41421356]] /n/n
